# Математическая статистика. Лабораторная работа №4. Васильев Н. А., Елисеев К. И.

# Задание №1

In [ ]:
!pip install -q gdown
import gdown

file_id = "1cx0pshptDSVmaWLJCBGS9jIIJ2g-VRgT"
file = "kc_house_data"

gdown.download(f"https://drive.google.com/uc?id={file_id}", file, quiet=False)

import pandas as pd

df = pd.read_csv(file)

Downloading...
From: https://drive.google.com/uc?id=1cx0pshptDSVmaWLJCBGS9jIIJ2g-VRgT
To: /content/kc_house_data
100%|██████████| 2.52M/2.52M [00:00<00:00, 132MB/s]


Линейная регрессия моделирует зависимость целевой переменной
\\(y\\) (цена недвижимости) от независимых переменных \\(x_1\\), \\(x_2\\), \\(x_3\\) (жилая площадь, soft_lot, soft_above) и свободного коэффициента \\(β_0\\).

Модель имеет вид:

$$
y=β_0+β_1x_1+β_2x_2+β_3x_3+ɛ
$$

где: \\(β_0\\) — свободный коэффициент (интерцепт), \\(β_1\\), \\(β_2\\), \\(β_3\\) — коэффициенты при независимых переменных, \\(ɛ\\) — случайная ошибка (нормально распределённая с нулевым средним) ($ε​∼\mathcal N(0,σ^2)$).

Параметры \\(β_0\\), \\(β_1\\), \\(β_2\\), \\(β_3\\) оцениваются методом наименьших квадратов (МНК), который минимизирует сумму квадратов остатков:

$$
RSS=\sum_{i=1}^{n}(y_i-\hat{y}_i)^2
$$

где \\(\hat{y}_i=β_0+β_1x_{1i}+β_2x_{2i}+β_3x_{3i}\\)

In [18]:
import numpy as np

X = df[['sqft_living', 'sqft_lot', 'sqft_above']].values
y = df['price'].values.reshape(-1, 1)

X_with_const = np.column_stack([np.ones(X.shape[0]), X])

XTX = np.dot(X_with_const.T, X_with_const)
XTX_inv = np.linalg.inv(XTX)
XTY = np.dot(X_with_const.T, y)
coefficients = np.dot(XTX_inv, XTY)

results = pd.DataFrame({
    'Коэффициент': coefficients.flatten()
}, index=['Intercept', 'sqft_living', 'sqft_lot', 'sqft_above'])

print("\nКоэффициенты:")
print(results)


Коэффициенты:
              Коэффициент
Intercept   -41445.123606
sqft_living    296.151642
sqft_lot        -0.278184
sqft_above     -16.903431


Остаточная дисперсия (несмещённая оценка дисперсии ошибок $\sigma^2$) вычисляется по формуле:

$$
\hat{\sigma}^2 = \frac{\text{RSS}}{n - k - 1},
$$

где: \\(RSS = \sum_{i=1}^n (y_i - \hat{y}_i)^2\\) — сумма квадратов остатков, \\(n\\) — количество наблюдений, \\(k\\) — количество независимых переменных (без учёта свободного коэффициента \\(\beta_0\\)), \\(\hat{y}_i\\) — предсказанные значения модели.

In [19]:
y_pred = np.dot(X_with_const, coefficients)

residuals = y - y_pred

rss = np.sum(residuals**2)

n = X.shape[0]
k = X.shape[1]
residual_variance = rss / (n - k - 1)

print(f"Остаточная дисперсия (σ²): {residual_variance:.2f}")

Остаточная дисперсия (σ²): 68179410013.81


Доверительные интервалы коэффициентов линейной регрессии

Для коэффициента $\beta_j$ доверительный интервал уровня $1-\alpha$ вычисляется по формуле:

$$
\hat{\beta}_j \pm t_{1-\alpha/2, n-k-1} \cdot \text{SE}(\hat{\beta}_j),
$$

где:
- $\hat{\beta}_j$ — МНК-оценка коэффициента,
- $t_{1-\alpha/2, n-k-1}$ — квантиль $t$-распределения,
- $\text{SE}(\hat{\beta}_j)$ — стандартная ошибка,
- $n$ — число наблюдений,
- $k$ — число регрессоров.

При этом Стандартная ошибка коэффициента

$$
\text{SE}(\hat{\beta}_j) = \sqrt{\hat{\sigma}^2 \cdot \left[ (\mathbf{X}^T\mathbf{X})^{-1} \right]_{jj}},
$$

где:
- $\hat{\sigma}^2 = \frac{1}{n-k-1}\sum_{i=1}^n (y_i - \hat{y}_i)^2$,
- $\mathbf{X}$ — матрица регрессоров,
- $n$ — число наблюдений,
- $k$ — число регрессоров.

In [22]:
from scipy.stats import t

n = X_with_const.shape[0]
k = X_with_const.shape[1] - 1

residuals = y - X_with_const @ coefficients
rss = np.sum(residuals**2)
sigma_squared = rss / (n - k - 1)

XTX_inv = np.linalg.inv(X_with_const.T @ X_with_const)
cov_matrix = sigma_squared * XTX_inv

std_errors = np.sqrt(np.diag(cov_matrix))

alpha = 0.05
t_critical = t.ppf(1 - alpha/2, df=n - k - 1)

conf_intervals = []
for i in range(len(coefficients)):
    coef = coefficients[i, 0]
    se = std_errors[i]
    lower = coef - t_critical * se
    upper = coef + t_critical * se
    conf_intervals.append((lower, upper))

coef_names = ['Intercept'] + ['sqft_living', 'sqft_lot', 'sqft_above']

print("Доверительные интервалы (95%):")
for name, interval in zip(coef_names, conf_intervals):
    print(f"{name}: [{interval[0]:.6f}, {interval[1]:.6f}]")

Доверительные интервалы (95%):
Intercept: [-50157.305600, -32732.941611]
sqft_living: [288.272297, 304.030988]
sqft_lot: [-0.363713, -0.192655]
sqft_above: [-25.659665, -8.147197]


Коэффициент детерминации \\( R^2 \\) определяется по формуле:

$$
R^2 = 1 - \frac{\sum_{i=1}^n (y_i - \hat{y}_i)^2}{\sum_{i=1}^n (y_i - \bar{y})^2}
$$

где:
- $y$ - вектор зависимой переменной,
- $̂y$ - вектор оценок зависимой переменной,
- $̄y$ – среднее значение зависимой переменной


In [23]:
y_pred = X_with_const @ coefficients

rss = np.sum((y - y_pred)**2)
tss = np.sum((y - np.mean(y))**2)

r_squared = 1 - rss / tss

print(f"R²: {r_squared:.4f}")

R²: 0.4942


**Проверим гипотезы:**



1.   Чем больше жилая площадь \\(x_1\\), тем больше цена \\(y\\).

Нулевая гипотеза: \\(H_0:β_1=0\\)

Альтернатива: \\(H_1:β_1>0\\)

Статистика t-теста:

$$
t=\frac{\hat{β}_1-0}{SE(\hat{β}_1)}
$$

где \\(SE(\hat{β}_1)\\) - стандартная ошибка коэффициента

Если \\(p < 0.05\\) и \\(\hat{β}_1>0\\), гипотеза \\(H_0\\) неверна.

2.   Цена зависит от \\(x_2\\) `sqft_lot`

Нулевая гипотеза: \\(H_0:β_2=0\\)

Альтернатива: \\(H_1:β_2\neq0\\)

Статистика t-теста:

$$
t=\frac{\hat{β}_2}{SE(\hat{β}_2)}
$$

Если \\(p < 0.05\\), гипотеза \\(H_0\\) неверна.

3.   Одновременное равенство нулю коэффициентов при \\(x_1\\) и \\(x_3\\)

Нулевая гипотеза: \\(H_0:β_1=β_3=0\\)

Альтернатива: \\(H_1:β_1 || β_3\neq0\\)

Статистика F-теста:

$$
F=\frac{(RSS_0-RSS)/q}{RSS/(n-k-1)}
$$

где \\(RSS_0\\) - остаточная сумма квадратов модели без \\(x_1\\) и \\(x_3\\), \\(RSS\\) - остаточная сумма квадратов полной модели, \\(q=2\\) - количество ограничений,  \\(k=3\\) - количество предикторов в полной модели.

Если \\(p < 0.05\\), гипотеза \\(H_0\\) неверна.

In [28]:
from scipy.stats import t, f

coef_names = ['Intercept', 'sqft_living', 'sqft_lot', 'sqft_above']
coef_dict = dict(zip(coef_names, coefficients.flatten()))
std_err_dict = dict(zip(coef_names, std_errors))

# 1. Проверка: Чем больше жилая площадь, тем больше цена
print("\nПроверка: Чем больше жилая площадь, тем больше цена")
coef = coef_dict['sqft_living']
se = std_err_dict['sqft_living']

t_stat = coef / se
p_value = 2 * (1 - t.cdf(np.abs(t_stat), df=n-k-1))

print(f"Коэффициент: {coef:.4f}, p-value: {p_value:.4f}")
if p_value < 0.05 and coef > 0:
    print("Подтверждено: жилая площадь влияет на цену пропорционально.")
else:
    print("Нет подтверждения.")

# 2. Проверка: Цена зависит от 'sqft_lot'
print("\nПроверка: Цена зависит от 'sqft_lot'")
coef = coef_dict['sqft_lot']
se = std_err_dict['sqft_lot']

t_stat = coef / se
p_value = 2 * (1 - t.cdf(np.abs(t_stat), df=n-k-1))

print(f"Коэффициент: {coef:.4f}, p-value: {p_value:.4f}")
if p_value < 0.05:
    print("Подтверждено: цена зависит от sqft_lot.")
else:
    print("Нет подтверждения.")

# 3. Проверка гипотезы H₀: β_sqft_living = β_sqft_above = 0
print("\nПроверка гипотезы H₀: β_sqft_living = β_sqft_above = 0")

rss_full = np.sum((y - X_with_const @ coefficients)**2)

X_restricted = np.column_stack([X_with_const[:, 0], X_with_const[:, 2]])
XTX_restricted = X_restricted.T @ X_restricted
coefficients_restricted = np.linalg.inv(XTX_restricted) @ X_restricted.T @ y
rss_restricted = np.sum((y - X_restricted @ coefficients_restricted)**2)

q = 2
f_stat = ((rss_restricted - rss_full)/q) / (rss_full/(n - k - 1))
p_value_f = 1 - f.cdf(f_stat, q, n-k-1)

print(f"F-статистика = {f_stat:.2f}, p-value: {p_value_f:.4f}")
if p_value_f < 0.05:
    print("Отвергаем H₀: хотя бы один коэффициент не равен нулю.")
else:
    print("Нет оснований отвергнуть H₀.")


Проверка: Чем больше жилая площадь, тем больше цена
Коэффициент: 296.1516, p-value: 0.0000
Подтверждено: жилая площадь влияет на цену положительно.

Проверка: Цена зависит от 'sqft_lot'
Коэффициент: -0.2782, p-value: 0.0000
Подтверждено: цена зависит от sqft_lot.

Проверка гипотезы H₀: β_sqft_living = β_sqft_above = 0
F-статистика = 10385.91, p-value: 0.0000
Отвергаем H₀: хотя бы один коэффициент не равен нулю.


# Задание №2

In [30]:
!pip install -q gdown
import gdown

file_id = "14L_y0LOAebuuqh8PllOw64cJQwVkmlV6"
file = "exams_dataset"

gdown.download(f"https://drive.google.com/uc?id={file_id}", file, quiet=False)

import pandas as pd

df = pd.read_csv(file)

Downloading...
From: https://drive.google.com/uc?id=14L_y0LOAebuuqh8PllOw64cJQwVkmlV6
To: /content/exams_dataset
100%|██████████| 72.0k/72.0k [00:00<00:00, 57.3MB/s]


Проверим гипотезу о равенстве средних на каждом уровне фактора с помощью модели однофакторного дисперсионного анализа.

Фактор - этническая/национальная группа `race/ethnicity`.

Выходная перемнная - суммарный балл за все три экзамена `math score` + `reading score` + `writing score`.

Гипотезы:

Нулевая гипотеза (средние значения во всех группах равны): \\(H_0:\mu_1=\mu_1=...=\mu_k\\)

Альтернатива: \\(H_1:∃i, j ∈ \{1, ..., k\} : \mu_i \neq \mu_j\\)

$$
F = \frac{MS_{межгрупповая}}{MS_{внутригрупповая}} = \frac{SS_{межгрупповая}/df_{межгрупповая}}{SS_{внутригрупповая}/df_{внутригрупповая}}
$$
$$
SS_{общая} = \sum_{i=1}^k \sum_{j=1}^{n_i} (y_{ij} - \bar{y})^2
$$
$$
SS_{межгрупповая} = \sum_{i=1}^k n_i (\bar{y}_i - \bar{y})^2
$$
$$
SS_{внутригрупповая} = \sum_{i=1}^k \sum_{j=1}^{n_i} (y_{ij} - \bar{y}_i)^2
$$
$$
df_{межгрупповая} = k - 1
$$
$$
df_{внутригрупповая} = N - k
$$

где:
- $k$ - число групп,
- $N$ - число наблюдений,
- $n_i$ - размер каждой группы,
- $\bar{y}_i$ - среднее в группе $i$,
- $\bar{y}$ - общее среднее

In [36]:
df['total_score'] = df[['math score', 'reading score', 'writing score']].sum(axis=1)

factor_col = 'race/ethnicity'
levels = df[factor_col].unique()
groups = [df[df[factor_col] == lvl]['total_score'].values for lvl in levels]

k = len(levels)
n_i = [len(g) for g in groups]
N = sum(n_i)
grand_mean = df['total_score'].mean()

ssb = sum(n * (g.mean() - grand_mean)**2 for n, g in zip(n_i, groups))
ssw = sum(((g - g.mean())**2).sum() for g in groups)

df_between = k - 1
df_within  = N - k

msb = ssb / df_between
msw = ssw / df_within

F_stat = msb / msw
p_value = 1 - f.cdf(F_stat, df_between, df_within)

print(f"Фактор «{factor_col}»:")
print("Уровни фактора:", levels)
print("Размеры групп:", n_i)
print("Средние по группам:", [float(round(g.mean(), 2)) for g in groups])
print(f"\nSSB = {ssb:.2f}, df_between = {df_between}")
print(f"SSW = {ssw:.2f}, df_within  = {df_within}")
print(f"MSB = {msb:.2f}, MSW = {msw:.2f}")
print(f"F = {F_stat:.3f}")
print(f"p-value = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print(f"\nПри α = {alpha:.2f} отвергаем нулевую гипотезу о равенстве средних.")
else:
    print(f"\nПри α = {alpha:.2f} недостаточно оснований отвергнуть нулевую гипотезу.")


Фактор «race/ethnicity»:
Уровни фактора: ['group B' 'group D' 'group A' 'group C' 'group E']
Размеры групп: [204, 261, 77, 324, 134]
Средние по группам: [195.06, 209.54, 191.66, 194.98, 223.81]

SSB = 112381.64, df_between = 4
SSW = 1953413.14, df_within  = 995
MSB = 28095.41, MSW = 1963.23
F = 14.311
p-value = 0.0000

При α = 0.05 отвергаем нулевую гипотезу о равенстве средних.
